In [16]:
import numpy as np
import torch
import torch.nn as nn

# Загрузка исходных данных
train_x_original = np.load('ihb.npy')  # [320, 10, 246]
train_y = np.load('train_y.npy')  # Метки (0 или 1)

# Заменяем NaN значения на 0.0
train_x_original = np.nan_to_num(train_x_original, nan=0.0)

# Преобразуем в тензоры PyTorch
train_x_tensor = torch.tensor(train_x_original, dtype=torch.float32)  # Признаки пар
train_y_tensor = torch.tensor(train_y, dtype=torch.float32)  # Метки (0 или 1)

# Создаем модель
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(246 * 2, 512)  # Входной слой
        self.fc2 = nn.Linear(512, 256)  # Второй слой
        self.fc3 = nn.Linear(256, 1)  # Выходной слой для бинарной классификации

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))  # Sigmoid для вывода в диапазоне [0, 1]
        return x

# Здесь предполагается, что модель уже обучена
model = SimpleNN()

# Функция для ручного тестирования
def manual_test(model, data_tensor):
    model.eval()  # Переводим модель в режим оценки (без обновления градиентов)
    
    # Запрашиваем у пользователя выбор субъекта
    subject_1 = int(input("Введите индекс первого объекта (0-319): "))
    subject_2 = int(input("Введите индекс второго объекта (0-319): "))
    
    # Выбираем временной шаг для обоих объектов
    timestep_1 = int(input("Введите индекс временного шага для первого объекта (0-9): "))
    timestep_2 = int(input("Введите индекс временного шага для второго объекта (0-9): "))
    
    # Извлекаем два набора признаков для выбранных субъектов и временных шагов
    input_1 = data_tensor[subject_1, timestep_1, :]  # Признаки первого объекта на временном шаге
    input_2 = data_tensor[subject_2, timestep_2, :]  # Признаки второго объекта на временном шаге
    
    # Объединяем их в один вектор (размер 492)
    combined_input = torch.cat((input_1, input_2), dim=0).unsqueeze(0)  # добавляем размерность для батча


    # Применяем модель
    with torch.no_grad():
        output = model(combined_input)
        print(f"Предсказанная метка модели: {output}")

# Запуск тестирования:
manual_test(model, train_x_tensor)

Предсказанная метка модели: tensor([[0.4992]])
